In [1]:
import torch
flag = torch.cuda.is_available()
print(flag)

if flag:
    print(torch.cuda.get_device_name(0))
    print(torch.rand(3,3).cuda()) 

True
NVIDIA RTX A5000
tensor([[0.6601, 0.0063, 0.1479],
        [0.9316, 0.0662, 0.5008],
        [0.3951, 0.6088, 0.6714]], device='cuda:0')


In [2]:
import sys
import os

# 确定项目根目录（假设当前工作目录是项目的根目录）
project_root = os.path.abspath("..")
sys.path.insert(0, project_root)

import pickle
from ai import ai_battle, MinimaxAI, MCTSAI
from game.chess_game import ChessGame
from CelestialVault.instances import ExampleThreadManager


class TrainDataThread(ExampleThreadManager):
    def get_args(self, obj: object):
        return (mcts_ai_0, mcts_ai_0, ChessGame((5, 5), 2), False)
    
    def process_result(self):
        all_training_data = []
        result_dict = self.get_result_dict()
        for d in result_dict:
            over_game = result_dict[d]
            history_board = over_game.history_board
            history_move = over_game.history_move
            for step in history_board:
                if step+1 not in history_board.keys():
                    continue
                board = self.process_board(history_board, step)
                all_training_data.append((board, history_move[step+1]))
        return all_training_data
    
    def process_board(self, history_board, step):
        color = 1 if step % 2 == 0 else -1
        
        processed_board = []
        for row in history_board[step]:
            processed_row = []
            for cell in row:
                processed_cell = cell + [color]
                if processed_cell[0] == float("inf"):
                    processed_cell[0] = 5
                processed_row.append(processed_cell)
            processed_board.append(processed_row)
        return processed_board



minimax_ai = MinimaxAI(3)
mcts_ai_0 = MCTSAI(1000, flag=True)
mcts_ai_1 = MCTSAI(1000, flag=False)

train_data_threader = TrainDataThread(
            ai_battle,
            thread_num=10,
            tqdm_desc='trainDataProcess',
            show_progress=True)


In [7]:

from time import strftime, localtime


def train_data(train_num):
    train_data_threader.start(range(train_num), "serial")
    train_data_threader.handle_error()
    all_training_data = train_data_threader.process_result()

    data_size = len(all_training_data)
    now_time = strftime("%m-%d-%H", localtime())
    pickle.dump(all_training_data, open(f"train_data/all_training_data({now_time})({data_size}).pkl", "wb"))

    return all_training_data

all_training_data = train_data(100)

all_training_data[0]

trainDataProcess: 100%|██████████| 10/10 [05:26<00:00, 32.66s/it]


([[[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]],
  [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]],
  [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]],
  [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]],
  [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]]],
 (2, 2))

In [5]:
with open("train_data/all_training_data(06-13-17)(1532).pkl", "rb") as f:
    all_training_data = pickle.load(f)

In [26]:
for board,_ in all_training_data:
    for row in board:
        for cell in row:
            if cell[0] == float("inf"):
                cell[0] = 5

In [8]:
import torch
from torch.utils.data import DataLoader, Dataset

class ChessDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        board_state, move = self.data[idx]
        board_state = torch.tensor(board_state, dtype=torch.float32)
        move = move[0] * 5 + move[1]
        return board_state, move

dataset = ChessDataset(all_training_data)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [9]:
from time import strftime, localtime
import torch.nn as nn
import torch.optim as optim
from ai.deeplearning import ChessModel, DeepLearningAI

# 设置CuDNN选项
torch.backends.cudnn.benchmark = True

model = ChessModel().cuda() # 初始化模型，并将其移动到GPU上
criterion = nn.CrossEntropyLoss() # 定义交叉熵损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001) # 定义Adam优化器

# 训练循环
num_epochs = 10  # 训练10个epoch
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        # 调整输入的维度，并将其移动到GPU上
        # inputs 的原始形状是 (batch_size, height, width, channels)，也就是 (32, 5, 5, 3)
        # inputs.permute(0, 3, 1, 2) 会将 inputs 的维度从 (32, 5, 5, 3) 转换为 (32, 3, 5, 5)
        inputs = inputs.permute(0, 3, 1, 2).cuda()  # (batch_size, channels, height, width)
        labels = labels.cuda()

        # 清零梯度
        optimizer.zero_grad()

        # 前向传播
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()

        # 累积损失
        running_loss += loss.item()
        if i % 100 == 99:  # 每100个batch打印一次loss
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')
now_time = strftime("%m-%d-%H", localtime())
torch.save(model.state_dict(), f'models/chess_ai_model({now_time}).pth')


Finished Training


In [4]:
from ai.deeplearning import DeepLearningAI

# 加载深度学习AI
deep_learning_ai = DeepLearningAI('models/chess_ai_model(06-13-23).pth')

# 与其他AI算法进行对战
# ai_battle(deep_learning_ai, minimax_ai, display=True)
ai_battle(deep_learning_ai, mcts_ai_0, ChessGame((5, 5), 2), display=True)



游戏开始！
蓝方AI: DeepLearningAI
红方AI: MCTSAI

tensor([[[-5.5624e+00,  6.5584e-01,  2.0341e+00,  1.5487e+00, -5.5546e+00],
         [-3.7541e+00,  2.4547e+00,  1.9769e+00,  1.5765e+00, -1.4523e+00],
         [-4.4398e+00,  1.9107e+00,  2.1068e+00,  3.1466e+00, -4.7992e-03],
         [-5.4730e+00,  1.1926e+00,  2.2297e+00,  1.5033e+00, -6.1840e+00],
         [-1.2070e+01, -7.1432e+00, -7.4015e+00, -2.3458e+00, -1.0167e+01]]])
tensor([[4.0553e-05, 2.0350e-02, 8.0750e-02, 4.9699e-02, 4.0868e-05],
        [2.4738e-04, 1.2297e-01, 7.6260e-02, 5.1098e-02, 2.4717e-03],
        [1.2461e-04, 7.1373e-02, 8.6841e-02, 2.4564e-01, 1.0511e-02],
        [4.4344e-05, 3.4807e-02, 9.8193e-02, 4.7493e-02, 2.1781e-05],
        [6.0515e-08, 8.3462e-06, 6.4460e-06, 1.0115e-03, 4.0569e-07]])
[
  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
  [[0, 0], [0, 0], [1, 1], [2, 2], [1, 1]],
  [[0, 0], [0, 0], [0, 0], [1, 1], [0, 0]],
  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
]
第1步